In [6]:
import torch
import catboost
from torch import nn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import TensorDataset,DataLoader
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.special import softmax
import warnings 
warnings.filterwarnings('ignore')
gc.collect()

29

In [7]:
seed = 5290

In [8]:
def backbone(self, x):
    x = self.module.conv1(x)
    x = self.module.bn1(x)
    x = self.module.relu(x)

    x = self.module.layer1(x)
    x = self.module.layer2(x)
    x = self.module.layer3(x)

    return x

def preprocessing_Y(file_path):
    Y = pd.read_csv(file_path).iloc[:,:]
    Y = Y[Y.Filename != 'train_01046']
    enc = OneHotEncoder().fit(Y[['Label']])
    Y_one_hot = enc.transform(Y[['Label']]).toarray()
    Y_one_hot = torch.FloatTensor(Y_one_hot)
    print('Y_ont_hot shape',Y_one_hot.shape)
    print('Y_df shape',Y.shape)
    return Y_one_hot,Y

def load_pretrain_senet(model_path):
    model = torch.hub.load(
        'moskomule/senet.pytorch',
        'se_resnet20',
        num_classes=6)
    model.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.fc = nn.Sequential(
    nn.Linear(64,64),
    nn.LeakyReLU(),
    nn.Dropout(0.5),#三個臭皮匠勝過一個諸葛亮
    nn.Linear(64,6))
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2 ,3])#平行運算模式
    model.load_state_dict(torch.load(model_path))#載入權重
    model.eval()#評估模式
    model.fc =  nn.Sequential(model.module.fc,nn.Softmax(dim=1))#取機率
    model.backbone = backbone.__get__(model)
    return model

def get_senet_output(senet,data):
    return senet(data)

def get_all_senet_output(data):
    y = senet(data)
    return y.detach().cpu().numpy()

def get_X_numpy(X):
    X_train_np = np.array([[]])
    data_iter = DataLoader(TensorDataset(torch.FloatTensor(X.to(torch.float32))),batch_size=256)
    for bx in tqdm(data_iter):
        bx = bx[0]
        bx = bx.to('cuda:0')
        y_hat = get_all_senet_output(bx)
        y_hat = softmax(y_hat,axis=1)
        if len(X_train_np) == 1:
            X_train_np = y_hat
        else:
            X_train_np = np.vstack((X_train_np,y_hat))
    return X_train_np

def get_X_numpy_backbone(X):
    X_train_np = np.array([[]])
    data_iter = DataLoader(TensorDataset(torch.FloatTensor(X.to(torch.float32))),batch_size=64)
    for bx in tqdm(data_iter):
        bx = bx[0]
        bx = bx.to('cuda:0')
        y_hat = senet.backbone(bx).detach().cpu().numpy()
        y_hat = y_hat.reshape(-1,64*20*40)
        if len(X_train_np) == 1:
            X_train_np = y_hat
        else:
            X_train_np = np.vstack((X_train_np,y_hat))
    return X_train_np

# 路徑

In [9]:
Y_train_path = 'train/meta_train.csv'
senet = 'senet20_2021-06-10-00-14-38_random_state5290_validacc_0.925_959訓練240驗證.pt'

# load k 個 senet

In [10]:
senet = load_pretrain_senet(senet).to('cuda:0')
senet

Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master


DataParallel(
  (module): CifarSEResNet(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): CifarSEBasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=16, out_features=1, bias=False)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=1, out_features=16, bias=False

# 訓練 和 測試資料

In [11]:
X_train_all = torch.load('X_train_吳啟聖教的方法.pt')
X_test_all = torch.load('X_test_吳啟聖教的方法.pt')
X_private_all = torch.load('X_private_test_吳啟聖教的方法.pt')

Y_train_all,Y_train_df = preprocessing_Y(Y_train_path)
map_dict = {}
for l in Y_train_df.Label.unique():
    map_dict[l] = Y_train_df[Y_train_df.Label==l].sample(1)['Remark'].values[0]
map_dict

Y_ont_hot shape torch.Size([1199, 6])
Y_df shape (1199, 3)


{0: 'Barking',
 1: 'Howling',
 2: 'Crying',
 3: 'COSmoke',
 4: 'GlassBreaking',
 5: 'Cat'}

In [12]:
'''
Y_test_all = pd.read_csv('vote_V12_排行榜0.976667.csv') #拿目前分數最高的
Y_test_all.head(5)
values = Y_test_all.iloc[:,1:].values.argmax(axis=1) #轉換成one hot格式
n_values = np.max(values) + 1
np.eye(n_values)[values][:5]
Y_test_all.iloc[:,1:] = np.eye(n_values)[values] #轉換成df格式
Y_test_all
Y_test_all = torch.FloatTensor(Y_test_all.iloc[:10000,1:].values) #轉換成tensor格式
Y_test_all,Y_test_all.shape
'''

"\nY_test_all = pd.read_csv('vote_V12_排行榜0.976667.csv') #拿目前分數最高的\nY_test_all.head(5)\nvalues = Y_test_all.iloc[:,1:].values.argmax(axis=1) #轉換成one hot格式\nn_values = np.max(values) + 1\nnp.eye(n_values)[values][:5]\nY_test_all.iloc[:,1:] = np.eye(n_values)[values] #轉換成df格式\nY_test_all\nY_test_all = torch.FloatTensor(Y_test_all.iloc[:10000,1:].values) #轉換成tensor格式\nY_test_all,Y_test_all.shape\n"

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, Y_train_all, 
                                                      test_size=0.2, 
                                                      random_state=seed,
                                                      stratify=Y_train_all)
print(X_train.shape)
print(X_valid.shape)
print(pd.DataFrame(y_train.argmax(axis=1)).value_counts())
print(pd.DataFrame(y_valid.argmax(axis=1)).value_counts())

torch.Size([959, 1, 80, 157])
torch.Size([240, 1, 80, 157])
0    160
1    160
2    160
3    160
5    160
4    159
dtype: int64
0    40
1    40
2    40
3    40
4    40
5    40
dtype: int64


In [15]:
from torch.utils.data import TensorDataset,DataLoader
trainset = TensorDataset(torch.FloatTensor(X_train.to(torch.float32)),torch.FloatTensor(y_train))
vaildset = TensorDataset(torch.FloatTensor(X_valid.to(torch.float32)),torch.FloatTensor(y_valid))
train_iter = DataLoader(trainset,batch_size=128,num_workers=4)
vaild_iter = DataLoader(vaildset,batch_size=128,num_workers=4)

In [16]:
device = 'cuda:0'
for bx,by in train_iter:
    print(bx.shape,bx.device) # batch,channel,freq,time
    print(by.shape,by.device) # batch,n_class
    print(senet.backbone(bx.to(device)).size())
    print(senet(bx[:3]))
    break

torch.Size([128, 1, 80, 157]) cpu
torch.Size([128, 6]) cpu
torch.Size([128, 64, 20, 40])
tensor([[-14.0959, -13.5227, -11.2204,  10.5843,  -9.0482,  -9.6640],
        [ -8.4355,  -3.2929,  -4.4402,  -3.5256,  -4.9135,   1.2465],
        [ -7.9972,  -5.7093,  -7.3221,  -7.8285,  -7.1900,   4.6640]],
       device='cuda:0', grad_fn=<GatherBackward>)


In [17]:
from sklearn.metrics import confusion_matrix,accuracy_score
def plot_confusion_matrix(model,data_iter,map_dict=map_dict):
    y_true = np.array([])
    y_pred = np.array([])
    for bx,by in tqdm(data_iter):
        bx = bx.to(device)
        by = by.to(device)
        y_hat = model(bx)
        try:
            if len(y_true) == 0:
                y_true = by.argmax(axis=1).detach().cpu().numpy()
                y_pred = y_hat.argmax(axis=1).detach().cpu().numpy()
            else:
                y_true = np.hstack((y_true,by.argmax(axis=1).detach().cpu().numpy()))
                y_pred = np.hstack((y_pred,y_hat.argmax(axis=1).detach().cpu().numpy()))
        except:
            pass
    cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
    cm.columns = list(map_dict.values())
    acc = accuracy_score(y_pred,y_true)
    return cm,acc

In [18]:
model = senet
model.eval()
cm,acc = plot_confusion_matrix(model.to(device),train_iter)
print(acc)
cm

100%|██████████| 8/8 [00:01<00:00,  5.76it/s]

0.9186652763295099


,Barking,Howling,Crying,COSmoke,GlassBreaking,Cat
0,138,4,3,1,0,0
1,9,149,18,1,0,1
2,3,3,127,0,0,1
3,2,1,10,158,4,0
4,3,0,0,0,153,2
5,5,3,2,0,2,156


In [19]:
cm,acc = plot_confusion_matrix(model.to(device),vaild_iter)
print(acc)
cm

100%|██████████| 2/2 [00:00<00:00,  2.07it/s]

0.925


,Barking,Howling,Crying,COSmoke,GlassBreaking,Cat
0,33,2,0,0,0,0
1,2,36,2,0,0,0
2,0,2,36,0,0,2
3,2,0,0,40,0,0
4,0,0,0,0,39,0
5,3,0,2,0,1,38


# 有辦法在提升驗證集的表現嗎?

# 接 LogisticRegression 修正senet的弱點

In [20]:
X_train_np = get_X_numpy(X_train)
y_train_np = y_train.detach().numpy().argmax(axis=1)
X_valid_np = get_X_numpy(X_valid)
y_valid_np = y_valid.detach().numpy().argmax(axis=1)
lg = LogisticRegression()
lg.fit(X_train_np,y_train_np)
y_true = y_valid_np
y_pred = lg.predict(X_valid_np )
print(accuracy_score(y_pred,y_true))
cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
cm.columns = list(map_dict.values())
cm

100%|██████████| 1/1 [00:00<00:00, 12.08it/s]


0.9208333333333333


,Barking,Howling,Crying,COSmoke,GlassBreaking,Cat
0,33,2,0,0,0,0
1,2,36,2,0,0,0
2,1,2,36,0,0,2
3,1,0,0,39,0,0
4,0,0,0,0,39,0
5,3,0,2,1,1,38


# cma-es優化 LogisticRegression

In [21]:
import optuna
sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler,direction='maximize')
search_iteration = 30
best_acc = 0
for epoch in tqdm(range(search_iteration)):
    trial = study.ask()
    class_weight = {}
    for i in [0,1,2,3,4,5]:
        class_weight[i] = trial.suggest_uniform(i,0,1)
    lg = LogisticRegression(class_weight=class_weight)
    lg.fit(X_train_np,y_train_np)
    y_true = y_valid_np
    y_pred = lg.predict(X_valid_np)
    acc = accuracy_score(y_pred,y_true)
    if acc>best_acc:
        best_acc = acc
    print(epoch,best_acc)
    study.tell(trial,acc)
    
study.best_params

[I 2021-06-10 00:17:14,244] A new study created in memory with name: no-name-a5727f3c-a20c-475e-ac48-e30cba7c8c00
 33%|███▎      | 10/30 [00:00<00:00, 46.11it/s]

0 0.9083333333333333
1 0.9125
2 0.9166666666666666
3 0.9208333333333333
4 0.9208333333333333
5 0.9208333333333333
6 0.9208333333333333
7 0.9208333333333333
8 0.9208333333333333
9 0.9208333333333333


 70%|███████   | 21/30 [00:00<00:00, 48.35it/s]

10 0.9208333333333333
11 0.9208333333333333
12 0.925
13 0.925
14 0.925
15 0.925
16 0.925
17 0.925
18 0.925
19 0.925
20 0.925


100%|██████████| 30/30 [00:00<00:00, 47.10it/s]

21 0.925
22 0.925
23 0.925
24 0.925
25 0.925
26 0.925
27 0.925
28 0.925
29 0.925


{0: 0.08324537800723988,
 1: 0.6026946650551065,
 2: 0.4983260703707777,
 3: 0.6081785177929264,
 4: 0.5034570042566819,
 5: 0.33452565319865263}

In [22]:
X_train_np = get_X_numpy(X_train)
y_train_np = y_train.detach().numpy().argmax(axis=1)
X_valid_np = get_X_numpy(X_valid)
y_valid_np = y_valid.detach().numpy().argmax(axis=1)
lg = LogisticRegression(class_weight=study.best_params)
lg.fit(X_train_np,y_train_np)
y_true = y_valid_np
y_pred = lg.predict(X_valid_np)
acc = accuracy_score(y_pred,y_true)
print(acc)
cm = pd.DataFrame(confusion_matrix(y_pred,y_true))
cm.columns = list(map_dict.values())
cm

100%|██████████| 1/1 [00:00<00:00, 26.50it/s]

0.925


,Barking,Howling,Crying,COSmoke,GlassBreaking,Cat
0,33,0,0,0,0,0
1,2,37,2,0,0,0
2,1,3,36,0,0,2
3,1,0,0,40,0,0
4,0,0,0,0,39,1
5,3,0,2,0,1,37


# 套用private_test

In [23]:
X_public_np = get_X_numpy(X_test_all)
X_private_np = get_X_numpy(X_private_all)
all_test_data = np.vstack((X_public_np,X_private_np))

100%|██████████| 79/79 [00:03<00:00, 23.41it/s]


In [24]:
all_test_data.shape

(30000, 6)

In [25]:
final_prob = lg.predict_proba(all_test_data)
final_prob.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [26]:
sample_submit = pd.read_csv('sample_submission.csv')
sample_submit.iloc[:30000,1:] = final_prob #三萬筆
sample_submit

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,0.017884,0.052423,0.038237,0.014810,0.030362,0.846284
1,public_00002,0.015033,0.040344,0.030961,0.011285,0.025663,0.876715
2,public_00003,0.156129,0.093246,0.168539,0.437445,0.086260,0.058381
3,public_00004,0.031156,0.104220,0.073902,0.026852,0.061237,0.702635
4,public_00005,0.039233,0.078124,0.634467,0.115604,0.078411,0.054160
...,...,...,...,...,...,...,...
29995,private_19996,0.007880,0.012527,0.034007,0.920264,0.016360,0.008962
29996,private_19997,0.092505,0.115340,0.125595,0.038888,0.078870,0.548802
29997,private_19998,0.012468,0.031175,0.024073,0.009012,0.021625,0.901648
29998,private_19999,0.020564,0.686627,0.237888,0.016004,0.019883,0.019034


In [27]:
print(sample_submit.shape)
print(sample_submit.sum(axis=1))

(30000, 7)
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
29995    1.0
29996    1.0
29997    1.0
29998    1.0
29999    1.0
Length: 30000, dtype: float64


In [28]:
acc

0.925

In [29]:
sample_submit.to_csv(f'submit_seed_{seed}_valid_acc_{acc}_senet_959訓練240驗證_接logistic做最後修正_預測private_test.csv',index=False)
print('done')

done
